In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType
from preproc_functions import preprocess_all
from pyspark.sql import functions as F
import pandas as pd
from preproc_functions import emoji_to_words

spark = SparkSession \
        .builder \
        .master('spark://10.10.28.172:7077') \
        .appName('joining_result_tables') \
        .enableHiveSupport() \
        .config("spark.pyfiles", "preproc_functions.py") \
        .getOrCreate()

# Change to 'OFF' if it produces too much output
spark.sparkContext.setLogLevel('OFF')  
# files used for preprocessing
spark.sparkContext.addPyFile("/home/ubuntu/twitter_sentiment/code/preprocessing/preproc_functions.py")

22/04/23 15:31:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [8]:
spark.sql('select * from twitter_data.raw_data where id = 1501510373228171265').show(truncate=False)

+-------------------+------------------------------------------------------------------+----------+-------------------+----+
|id                 |text                                                              |author_id |created_at         |geo |
+-------------------+------------------------------------------------------------------+----------+-------------------+----+
|1501510373228171265|Russia invades Ukraine: Live updates - CNN https://t.co/vVPuPfjJP6|4505300234|2022-03-09 10:49:04|null|
+-------------------+------------------------------------------------------------------+----------+-------------------+----+



In [4]:
# name of database
spark.sql('use twitter_data')

query = '''
    SELECT *
    FROM raw_data
    WHERE text IS NOT NULL
    AND created_at IS NOT NULL
    AND id IS NOT NULL
'''

raw_data = spark.sql(query)

In [5]:
udf_preprocessing = udf(lambda text: preprocess_all(text), StringType())
raw_data = raw_data.withColumn('text', udf_preprocessing(col('text')))

In [6]:
data = raw_data.select(['id', 'text'])

### 
1. translates emojies
2. removes user mentions
3. removes urls
4. removes all non-numerical characters
5. all text to lowercase

In [7]:
udf_translate_emojies = udf(lambda text: emoji_to_words(text))
user_regex = r'(@\w{1,15})'
url_regex = r'(https?:\/\/)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)'
neg_alphanumeric_regex = r'([^ a-zA-Z])'
double_spaces_regex = r'(  +)'
start_spaces_regex = r'(^ )'

data.\
    withColumn('text', udf_translate_emojies(col('text'))). \
    withColumn('text', F.regexp_replace(col('text'), url_regex, '')). \
    withColumn('text', F.regexp_replace(col('text'), user_regex, '')). \
    withColumn('text', F.regexp_replace(col('text'), neg_alphanumeric_regex, '')). \
    withColumn('text', F.regexp_replace(col('text'), double_spaces_regex, ' ')). \
    withColumn('text', F.regexp_replace(col('text'), start_spaces_regex, '')). \
    withColumn('text', F.lower(col('text'))
).show(5)


+-------------------+--------------------+
|                 id|                text|
+-------------------+--------------------+
|1486390083485843460|ive deleted my tw...|
|1486390081246085125|oil hits highest ...|
|1486390080956944385|did tucker carlso...|
|1486390059498713099|west supports lar...|
|1486390057896456200|russia wont invad...|
+-------------------+--------------------+
only showing top 5 rows



In [10]:
spark.sql('select count(*) from textblob_results').show()
spark.sql('select count(*) from vader_results').show()
spark.sql('select count(*) from wordlist2477_results').show()

+--------+
|count(1)|
+--------+
| 9023607|
+--------+

+--------+
|count(1)|
+--------+
| 9023607|
+--------+

+--------+
|count(1)|
+--------+
| 9023607|
+--------+



In [8]:
joined_results.write.mode("overwrite").saveAsTable("results")

KeyboardInterrupt: 

In [ ]:
joined_results.show(5)